In [5]:
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from PIL import Image
import os
from pathlib import Path
from utils import *
from allResNets import *
from engine import *
from data_transforms import create_transform
from dataloaders import *

change_to_disk()
data_dir = Path("clean_split_1k/")
models_path = Path('Quality/Models_test/')
check_dir(models_path)

In [6]:
def find_classes(directory: str) -> Tuple[List[str], Dict[str, int]]:
    """Finds the class folder names in a target directory."""

    # 1. Get the class names by scanning the target directory
    classes = sorted(entry.name for entry in os.scandir(directory) if entry.is_dir())

    # 2. Raise an error if class names could not be found
    if not classes:
        raise FileNotFoundError(f"Couldn't find any classes in {directory}... please check file structure.")

    # 3. Create a dictionary of index labels (computers prefer numbers rather than strings as labels)

    class_to_idx = {}
    for i, class_name in enumerate(classes):
        id = np.zeros(len(classes))
        id[i] = 1
        class_to_idx[class_name] = id
        
    return classes, class_to_idx

class CustomImageFolderMultiLabel(torch.utils.data.Dataset):
    def __init__(self, root, transform = None):
        self.root = root
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes, self.class_to_idx = find_classes(root)
        for i, class_name in enumerate(self.classes):
            class_path = os.path.join(root, class_name)
            for image_path in os.listdir(class_path):
                self.image_paths.append(os.path.join(class_path, image_path))
                label = [0]*len(self.classes)
                label[i] = 1
                self.labels.append(torch.tensor(label))

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.image_paths)
    


NUM_WORKERS = os.cpu_count()
def create_dataloaders_multilabel(
    data_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers:int = NUM_WORKERS):

    """
    Create dataloaders for the data split into train/validation/test

    Args:
    data_dir : Path to data directory with (train/validation/test split).
    transform : torchvision transforms to perform on training, validation and testing data.
    batch_size : Number of samples per batch in each of the DataLoaders.
    num_workers : An integer for number of workers per DataLoader.

    Returns:
    
    train dataloader, validation_dataloader, test_dataloader, train_data, validation_data, test_data, class_names
    """

    train_dir = data_dir / "train/"
    validation_dir = data_dir / "validation/"
    test_dir = data_dir / "test/"


    # Use ImageFolder to create dataset(s)
    train_data = CustomImageFolderMultiLabel(root=train_dir,
                                    transform=transform) # a transform for the data
    
    validation_data = CustomImageFolderMultiLabel(root=validation_dir,
                                    transform=transform)

    test_data = CustomImageFolderMultiLabel(root=test_dir,
                                    transform=transform)


    #Get classes
    class_names = train_data.class_to_idx

    # Turn images into data loaders
    train_dataloader = DataLoader(dataset=train_data,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=True)

    test_dataloader = DataLoader(dataset=test_data,
                                batch_size=batch_size,
                                num_workers=num_workers,
                                shuffle=False)

    validation_dataloader = DataLoader(dataset=validation_data,
                                        batch_size=batch_size,
                                        num_workers=num_workers,
                                        shuffle=False)

    return train_dataloader, validation_dataloader, test_dataloader, train_data, validation_data, test_data, class_names

In [8]:
image_shape = (1080, 1920)
resize_factor = 0.1
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]
BATCH_SIZE = 16

data_transform = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=True, gauss_noise=True)

train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders_multilabel(data_dir, transform = data_transform, batch_size=BATCH_SIZE, num_workers= NUM_WORKERS)

In [9]:
NUM_EPOCHS = 2
lr = 0.001
device = get_device()

resnet18 = ResNet18()

#Changed in order to fit the multilabel problem
loss_fn = nn.BCEWithLogitsLoss()

In [10]:
model_name = "ResNet18"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet18
resnet18.to(device)
nadam_optim = torch.optim.NAdam(params=resnet18.parameters(), lr=lr)
train_resnet18_results, train_time_resnet18 = train(resnet18, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

Quality\Models_test\ResNet18 does not exist, creating one...


  0%|          | 0/2 [00:00<?, ?it/s]